## PyTorch 튜토리얼

* Tensor : Numpy의 ndarray와 유사하나 GPU에서도 실행이 가능.

In [1]:
import torch
import numpy as np

#### Tensor 초기화

In [2]:
# 2차원 List -> Tensor
data = [[1, 2], [3, 4]]

# Tensor
x_data = torch.tensor(data)

print(x_data)
print(x_data.shape)

tensor([[1, 2],
        [3, 4]])
torch.Size([2, 2])


In [3]:
# Random값으로 Tensor 생성. 정수값으로 Tensor의 차원을 전달
tensor = torch.rand(3, 4)
print("tensor:", tensor)
print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

tensor: tensor([[0.7794, 0.1118, 0.5912, 0.5936],
        [0.4897, 0.3600, 0.0690, 0.8786],
        [0.5208, 0.4229, 0.0316, 0.3188]])
Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [4]:
# Returns a tensor (filled with the scalar value 1)
tensor = torch.ones(4, 4) # the shape defined by the argument size
print(tensor)

# 모든 행에 대해 1번열에 0을 대입
tensor[:, 1] = 0
print(tensor)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [5]:
# Concatenate : 텐서를 연결해 입력으로 사용하기 --> 두 텐서에 담긴 정보를 모두 사용
# dim : 텐서를 연결해 어느 차원을 늘릴 것인지를 표시
t1 = torch.cat([tensor, tensor], dim=0) # 두 텐서를 연결해 0번째 차원을 늘리기
print("tensor shape:", tensor.shape)
print("->", t1)
print("t1 shape:", t1.shape) # 0번째 dimension이 늘어난 것을 확인
print("---------------------------")

t2 = torch.cat([tensor, tensor], dim=1) # 두 텐서를 연결해 1번째 차원을 늘리기
print("tensor shape:", tensor.shape)
print("->", t2)
print("t2 shape:", t2.shape) # 1번째 dimension이 늘어난 것을 확인

tensor shape: torch.Size([4, 4])
-> tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
t1 shape: torch.Size([8, 4])
---------------------------
tensor shape: torch.Size([4, 4])
-> tensor([[1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1.]])
t2 shape: torch.Size([4, 8])


## 신경망 모델 생성하기

* torch.nn 패키지
* torch.nn.Module : 모든 신경망의 Base Class
  * 새로운 신경망 모델은 해당 클래스를 상속해서 정의
  * 클래스 내에서 \__init()__, forward() 함수를 반드시 override.

\__init()__ : module, activation 등을 정의

forward : 모델에서 실행되어야 하는 연산 정의

In [6]:
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    # Input Image channel 1
    # Output Image channel 6 
    # 5 * 5 Convolution Matrix
    # nn.Conv2d(in_channels, out_channels, kernel_size, stride=1)
    self.conv1 = nn.Conv2d(1, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)

    # Fully Connected Layer
    self.fc1 = nn.Linear(16 * 5 * 5, 120) # 5 * 5 : 이미지 차원
    self.fc2 = nn.Linear(120, 60)
    self.fc3 = nn.Linear(60, 10)

  def forward(self, x):
    # Input = (32 * 32)
    # conv1을 통과 = (28 * 28)
    # MaxPool(2, 2) = (14 * 14) 
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # [batch size, 채널 크기, 이미지 가로, 세로 크기]

    # conv2를 통과 = (10 * 10)
    # MaxPool(2, 2) = (5 * 5)
    x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2)) # [1, 16, 5, 5]

    # 0번째 원소인 batch를 제외한 모든 차원을 하나로 평탄화
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x)) # [1, 120]
    x = F.relu(self.fc2(x)) # [1, 60]
    x = self.fc3(x) # [1, 10]
    return x

In [8]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=10, bias=True)
)


In [9]:
input = torch.randn(1, 1, 32, 32) # [batch size, 입력 채널, 가로, 세로]
out = net(input)
print("out shape:", out.shape)
print(out)

out shape: torch.Size([1, 10])
tensor([[-0.0372,  0.0612, -0.0754,  0.0389, -0.0850,  0.0688,  0.0067,  0.0358,
         -0.1118, -0.1103]], grad_fn=<AddmmBackward0>)


In [10]:
# view : reshape 역할. Tensor의 shape을 변경. (원소 수는 유지)
# view(1, -1) : 첫번째 차원은 1. -1로 표시된 2번째 차원은 알아서 계산
output = net(input)
print("output shape:", output.shape)
target = torch.randn(10) # 임의의 텐서를 정답값으로 가정
print("target shape:", target.shape)
target = target.view(1, -1) # 모델의 출력 텐서와 동일한 shape로 변경 (1, 10)
print("After reshape(view) -> target shape:", target.shape)

output shape: torch.Size([1, 10])
target shape: torch.Size([10])
After reshape(view) -> target shape: torch.Size([1, 10])


#### Loss Function 설정

In [11]:
import torch
import torch.nn as nn

# 학습에 필요한 Tensor를 backpropagation을 통해 gradient를 구하기
# 옵션 requires_grad = True : Tensor에 실행되는 모든 연산들을 트래킹. 자동으로 gradient를 계산
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)

# MSE 설정
criterion = nn.MSELoss()
output = criterion(input, target) # Loss 계산
output.backward()
print("input: ", input)
print("target: ", target)
print("output: ", output)

input:  tensor([[-0.1783,  0.6461,  0.4232,  0.3316,  1.3116],
        [ 0.4939,  0.2297,  1.1208,  0.8609,  0.3326],
        [-0.5029, -0.6107, -0.5427, -1.0697, -0.2754]], requires_grad=True)
target:  tensor([[ 0.8685, -0.0312, -0.0097, -0.0281,  0.6138],
        [-0.0686,  1.6287, -0.2403,  1.1914, -1.1053],
        [ 0.9482,  0.0656,  1.3955,  1.2405, -1.1778]])
output:  tensor(1.4088, grad_fn=<MseLossBackward0>)


#### Optimizer 설정

In [12]:
import torch.optim as optim

# Adam 객체 생성
optimizer = optim.Adam(net.parameters(), lr=0.01)

# 학습을 시작하기 전 gradients를 zero로 reset
optimizer.zero_grad()

input = torch.randn(1, 1, 32, 32)
output = net(input)
target = torch.randn(10) # 임의의 정답
target = target.view(1, -1)
loss = criterion(output, target) # Loss 계산
print(loss)

loss.backward()
optimizer.step()

tensor(1.5212, grad_fn=<MseLossBackward0>)


#### GPU 가속 이용하기

In [13]:
print(torch.cuda.is_available())

False


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=10, bias=True)
)

#### PyTorch TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [15]:
# Random한 Train data 생성
train_inputs = torch.randn(100, 32, 128) # [전체 데이터, 데이터의 최대 길이, hidden vector 크기]
train_labels = torch.randn(100, 3) # [전체 데이터 개수, 예측할 클래스 개수]

In [16]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [17]:
batch_size = 12

train_data = TensorDataset(train_inputs, train_labels) # 학습데이터, 레이블을 하나의 TensorDataset으로 결합
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [18]:
for step, batch in enumerate(train_dataloader):
  x_data, x_label = batch

  # 데이터 로딩 step
  print(step, x_data.shape, x_label.shape)

0 torch.Size([12, 32, 128]) torch.Size([12, 3])
1 torch.Size([12, 32, 128]) torch.Size([12, 3])
2 torch.Size([12, 32, 128]) torch.Size([12, 3])
3 torch.Size([12, 32, 128]) torch.Size([12, 3])
4 torch.Size([12, 32, 128]) torch.Size([12, 3])
5 torch.Size([12, 32, 128]) torch.Size([12, 3])
6 torch.Size([12, 32, 128]) torch.Size([12, 3])
7 torch.Size([12, 32, 128]) torch.Size([12, 3])
8 torch.Size([4, 32, 128]) torch.Size([4, 3])
